In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Who are you?

Answer: I am the LORD, your God.

Exodus 3:14-15 (NIV)

In this passage, Moses is tending his father-in-law's flock when he comes across a burning bush that is not consumed by flames. When he approaches the bush to investigate, God reveals himself to Moses in a theophany - a visible manifestation of God's presence. In response to Moses' question about who he is, God identifies himself as "I AM WHO I AM," or Yahweh, which means "he who exists forever." This name emphasizes God's eternal nature and highlights his unique identity as the one who has always existed and will always exist.

The phrase "I AM" is also significant because it echoes the Hebrew verb "to be," which is used throughout the Bible to express existence or reality. By using this verb, God is affirming his own existence and emphasizing that he is the source of all being and reality.

In addition to its literal meaning, the name "Yahweh" has been interpreted in various ways throughout history. Some have seen 

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

360

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1324

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the text, LLaMA-2 was trained on 2 trillion tokens of data.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  According to the text, LLaMA-2 was trained on 2 trillion tokens of data.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###Mamba Questions

In [ ]:
# Mamba Questions

questions_file_path = '/content/questions/MambaQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba achieves computational efficiency
without specialized hardware optimizations through its use of selective state space models (SSMs). SSMs dynamically adjust model parameters based on
the output, allowing for real-time processing capabilities and efficient computation.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the passage, Mamba introduces selective
state space models to manage long sequence data processing. These models dynamically adjust model parameters based on the output, optimizing real-time
processing capabilities and adjusting computational focus dynamically.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Based on the given passage,
Mamba's performance evaluation suggests several areas for future research and development:

1. Longer sequences and more complex datasets: While preliminary evaluations suggest that Mamba can perform competently on shorter sequences and
simple datasets, there is a need to explore its efficacy on longer sequences and more complex datasets.
2. Broader hardware optimization: Although Mamba operates effectively under standard GPU configurations, there is potential for broader hardware
optimization to enhance its accessibility and performance.
3. Expanding the evaluation of Mamba: Currently, the evaluation of Mamba is limited to specific tasks such as video processing and short text
snippets. There is a need to expand the evaluation to other applications and datasets to fully assess its capabilities.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Dynamic parameter adjustment plays a
crucial role in Mamba's selective state space models. The model uses a dynamic approach to adjust parameters based on the current state of the system
being modeled. This allows the model to adapt to changing conditions and improve its predictions over time. Specifically, Mamba's model adjusts
parameters in response to changes in the variance of the system, allowing it to capture complex dynamics and nonlinear relationships between
variables. By continuously updating parameters based on new data, Mamba's model can maintain high accuracy and precision in its predictions, even in
the presence of uncertainty and noise.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Mamba addresses the
inefficiencies of Transformers in moderate to long sequence processing by simplifying existing neural network architectures and reducing the
dependency on traditional attention mechanisms. It integrates RNN-like and CNN-like layers, reintroduces MLP blocks, and dynamically adjusts model
parameters based on output to enhance real-time processing capabilities. Unlike conventional approaches, Mamba operates under standard GPU
configurations without requiring specialized hardware adaptations, making it more computationally efficient.


Sources:
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
/content/papers/Mamba.pdf
  The design of Mamba has
significant implications for its applicability across different data modalities. Firstly, the use of structured state space models (SSMs) allows Mamba
to handle a broader range of applications beyond just video processing, including short text snippets. This is because SSMs 

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How does Mamba achieve computational efficiency without specialized hardware optimizations?
 response:   Mamba achieves computational efficiency
without specialized hardware optimizations through its use of selective state space models (SSMs). SSMs dynamically adjust model parameters based on
the output, allowing for real-time processing capabilities and efficient computation.


Q1: What innovations does Mamba introduce to manage long sequence data processing?
 response:   According to the passage, Mamba introduces selective
state space models to manage long sequence data processing. These models dynamically adjust model parameters based on the output, optimizing real-time
processing capabilities and adjusting computational focus dynamically.


Q2: How does the selective state space model in Mamba differ from traditional attention mechanisms in handling sequence data?
 response:   The
selective state space model in Mamba differs from traditional attention mechanisms in handling seq

In [ ]:
json_file_name = 'MambaQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'MambaQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Parametric Magnon Questions

In [ ]:
# Parametric Magnon Questions

questions_file_path = '/content/questions/ParametricMagnonQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Based on the given text, the materials
typically used to construct quantum transducers in hybrid quantum systems are:

1. Yttrium iron garnet (YIG)
2. Diamond
3. Silicon carbide (SiC)


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Magnon nonlinearities enhance quantum transduction by allowing for
selective tuning of the spin-magnon coupling "on" and "off." This control over the coupling enables protection of the spin centers against any
resonant magnon noise-induced decoherence.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Parametric magnonics introduces an innovative
approach in quantum transducers by utilizing nonlinear magnon dynamics to enhance microwave transduction to spin qubits. Unlike traditional hybrid
systems that rely on linearly-excited magnons, the proposed transducer exploits parametrically generated magnons to selectively tune the spin-magnon
coupling "on" and "off." This provides a new means of controlling the coupling between the spin centers and magnons, which could potentially protect
the spin centers against resonant magnon noise-induced errors.


Sources:
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Introducing nonlinear magnonics into quantum computing
systems can significantly impact their functionality and capabilities. Nonlinear magnonics refers to the study of the nonlinear properties of magnonic
materials, which can be used to manipulate and control magnon waves in ways that are not possible with linear magnonics. By incorporating nonlinear
magnonics into quantum computing systems, it may be possible to achieve new levels of performance and efficiency, including:

1. Enhanced coupling between magnonic and quantum systems: Nonlinear magnonics can enable stronger couplings between magnonic and quantum systems,
leading to more efficient transfer of information and greater stability in quantum computations.
2. New types of quantum gates: Nonlinear magnonics can allow for the creation of new types of quantum gates that take advantage of the nonlinear
properties of magnonic materials. These gates could potentially offer improved fidelity and faster operation times compared to tr

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications that could benefit from
magnon nonlinearities in quantum systems include:

1. Enhanced quantum computing: Nonlinear magnonics could be used to create more powerful and efficient quantum computers by leveraging the unique
properties of magnon nonlinearities.
2. Quantum communication: Magnon nonlinearities could be used to improve the performance of quantum communication systems, such as quantum key
distribution and quantum teleportation.
3. Quantum metrology: Nonlinear magnonics could be used to enhance the precision of quantum metrology techniques, such as magnetometry and
gravitational wave detection.
4. Quantum simulation: Magnon nonlinearities could be used to simulate complex quantum systems that are difficult to model using traditional
computational methods.
5. Quantum control: Nonlinear magnonics could be used to develop new methods for controlling and manipulating quantum systems, which could lead to
breakthroughs in areas such as quantum chemistry and quan

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What materials are typically used to construct quantum transducers in hybrid quantum systems?
 response:   Based on the given text, the materials
typically used to construct quantum transducers in hybrid quantum systems are:

1. Yttrium iron garnet (YIG)
2. Diamond
3. Silicon carbide (SiC)


Q1: How do magnon nonlinearities enhance quantum transduction?
 response:   Magnon nonlinearities enhance quantum transduction by allowing for
selective tuning of the spin-magnon coupling "on" and "off." This control over the coupling enables protection of the spin centers against any
resonant magnon noise-induced decoherence.


Q2: What techniques are used to minimize the microwave footprint in quantum computing transducers?
 response:   To minimize the microwave footprint
in quantum computing transducers, several techniques are employed, including:

1. Parametric magnon effects: By downconverting the microwave driving frequency using parametrically excited magnons, the footprint can be
signif

In [ ]:
json_file_name = 'ParametricMagnonQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'ParametricMagnonQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Quantum Mechanics Questions

In [ ]:
# Quantum Mechanics Questions

questions_file_path = '/content/questions/QuantumMechanicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics has had a profound impact on modern
technology, particularly in the fields of computing, cryptography, and sensing. Here are some key ways in which quantum mechanics has influenced
technology:

1. Superfast quantum computers: Quantum computers have the potential to solve complex problems much faster than classical computers. This is because
quantum bits (qubits) can exist in multiple states simultaneously, allowing them to process vast amounts of data quickly.
2. Unbreakable quantum cryptography: Quantum cryptography provides secure communication channels that are resistant to eavesdropping. This is because
any attempt to measure a quantum system will disturb its state, making it detectable if someone is trying to intercept the communication.
3. Ultrasensitive quantum sensors: Quantum sensors can measure incredibly small changes in physical quantities, such as magnetic fields or
temperature. This makes them ideal for applications like medical imaging, navigation, an

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Quantum mechanics has greatly influenced the creation of new fields of
study, particularly in the areas of quantum information theory and quantum thermodynamics. These fields have emerged as a result of the improved
understanding of the resource power of quantum phenomena, which has triggered a technological overhaul rivaling the three major industrial revolutions
of the past century. Quantum mechanics has also led to the development of novel mathematical and computational tools applicable to other domains,
including condensed matter physics, statistical mechanics, and cosmology.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  According to the text, some of the fundamental
questions about quantum mechanics that remain open include:

* The interpretation of quantum mechanics' elusive foundations
* The nature of time
* Causality
* Quantum measurement
* Quantum randomness
* Non-locality
* Particle indistinguishability

These questions are still being debated and researched by the international community of quantum scientists and engineers, who are working to shed
light on the physical meaning of fundamental quantum principles and to push the boundaries of the quantum description of the world.


Sources:
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf
/content/papers/quantum.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Exploring quantum mechanics contributes to theoretical
physics by providing insights into the fundamental nature of matter and energy at the smallest scales. Quantum mechanics is a mathematical framework
that describes the behavior of particles and systems at these scales, and it has been incredibly successful in explaining a wide range of phenomena,
from the properties of atoms and molecules to the behavior of solids and liquids. However, despite its successes, there are still many open questions
and uncertainties in quantum mechanics, particularly when it comes to its foundations.

By exploring these foundations, researchers can gain a deeper understanding of the underlying laws of physics and how they relate to the observed
behavior of quantum systems. This can lead to new discoveries and breakthroughs, as well as improvements in the accuracy and predictive power of
quantum mechanics. Additionally, the insights gained from studying quantum mechanics can have practical applications

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental advances in verifying quantum theory include:

1. Quantum computing: Developing and testing quantum computers to demonstrate their ability to perform certain calculations exponentially faster than
classical computers.
2. Quantum simulation: Using quantum systems to simulate complex quantum phenomena, such as chemical reactions or many-body systems, which are
difficult to study classically.
3. Quantum metrology: Demonstrating the use of quantum systems for precise measurements, such as timekeeping or magnetometry, beyond what is possible
with classical instruments.
4. Quantum communication: Developing secure quantum communication protocols, such as quantum key distribution, to protect against eavesdropping and
ensure the security of sensitive information.
5. Quantum cryptography: Developing and testing quantum cryptographic methods, such as quantum teleportation, to enable secure transmission of quantum
information over long distances.
6. Quantum optics: Investigating the

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the significance of quantum mechanics in modern technology?
 response:   Quantum mechanics has had a profound impact on modern
technology, particularly in the fields of computing, cryptography, and sensing. Here are some key ways in which quantum mechanics has influenced
technology:

1. Superfast quantum computers: Quantum computers have the potential to solve complex problems much faster than classical computers. This is because
quantum bits (qubits) can exist in multiple states simultaneously, allowing them to process vast amounts of data quickly.
2. Unbreakable quantum cryptography: Quantum cryptography provides secure communication channels that are resistant to eavesdropping. This is because
any attempt to measure a quantum system will disturb its state, making it detectable if someone is trying to intercept the communication.
3. Ultrasensitive quantum sensors: Quantum sensors can measure incredibly small changes in physical quantities, such as magnetic fields or
tempe

In [ ]:
json_file_name = 'QuantumMechanicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'Quantum MechanicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Qubit Teleportation Questions

In [ ]:
# Qubit Teleportation Questions

questions_file_path = '/content/questions/QubitTeleportationQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the article, the main components used in
quantum networks for teleportation are:

1. NV centres (electronic spins) in diamonds, which are used as communication qubits.
2. Nearby 13C nuclear spins, which are used as memory qubits.
3. Optical connections between nodes, which enable the transmission of quantum information.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Entanglement is established between distant nodes in a
quantum network through a process called entanglement swapping. This involves using a middle node, which is already entangled with one of the distant
nodes, to create a shared entangled state between the other two distant nodes.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory qubits play a crucial role in the
process of quantum teleportation in a network. They are used to store the quantum state of the qubit being teleported, allowing it to be transmitted
accurately across non-neighboring nodes in the network. The memory qubits are entangled with the communication qubit, which is used to generate remote
entanglement between the neighboring nodes. This allows the quantum state to be teleported from one node to another without physically moving it.
Additionally, the memory qubits are protected from decoherence during entanglement generation, ensuring that the teleported qubit state is preserved
with high fidelity.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Future applications of quantum teleportation in
quantum networks include:

1. Multi-node protocols: Teleportation-based protocols can be used to transfer quantum information between multiple nodes in a quantum network,
enabling the distribution of quantum entanglement over long distances.
2. Quantum cryptography: Quantum teleportation can be used to securely transmit encrypted quantum keys between distant nodes in a quantum network,
providing a robust foundation for quantum cryptography.
3. Quantum computing: Quantum teleportation can be used to transfer quantum information between different parts of a quantum computer, allowing for
more efficient computation and potentially enabling the development of larger-scale quantum computers.
4. Quantum metrology: Quantum teleportation can be used to distribute quantum information between different nodes in a quantum network, enabling more
accurate measurements and potentially leading to breakthroughs in fields such as gravitational wave dete

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Real-time feed-forward significantly impacts the
process of quantum teleportation by enabling unconditional teleportation. With real-time feed-forward, the state transfer is achieved each time a
qubit state is inserted into the teleporter, regardless of whether the feed-forward gate operations were applied on Alice. This allows for
unconditional teleportation between non-connected nodes in the network, which was previously prevented due to the demanding set of requirements on the
pre-shared entanglement, the BSM, and the coherence times.


Sources:
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
/content/papers/Qubit teleportation.pdf
  Technical
advancements needed to realize efficient quantum teleportation between non-neighboring nodes include:

1. Establishing remote entanglement on the two links between non-neighboring nodes.
2. Entanglement swapping on the middle node 

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What are the main components used in quantum networks for teleportation?
 response:   According to the article, the main components used in
quantum networks for teleportation are:

1. NV centres (electronic spins) in diamonds, which are used as communication qubits.
2. Nearby 13C nuclear spins, which are used as memory qubits.
3. Optical connections between nodes, which enable the transmission of quantum information.


Q1: How is entanglement established between distant nodes in a quantum network?
 response:   Entanglement is established between distant nodes in a
quantum network through a process called entanglement swapping. This involves using a middle node, which is already entangled with one of the distant
nodes, to create a shared entangled state between the other two distant nodes.


Q2: What role does the Bell-state measurement (BSM) play in quantum teleportation?
 response:   The Bell-state measurement (BSM) plays a crucial role
in quantum teleportation. In the protocol de

In [ ]:
json_file_name = 'QubitTeleportationQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QubitTeleportationQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Variance Based Questions

In [ ]:
# Variance Based Questions

questions_file_path = '/content/questions/VarianceQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The focus of variance-based sensitivity analysis in
quantum memory is todetermine the sensitivity of a quantum memory implementation toexperimental fluctuations and drift, particularly in terms of
itsperformance metric, such as memory efficiency.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In the context of the paper " Variance-based sensitivity analysis of /Lambda1-type quantum
memory," /Lambda1-type quantum memory refers to a specific type of optical quantum memory that utilizes the /Lambda1 resonance transition in an atomic
ensemble to store and retrieve photonic quantum states.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Memory sensitivity analysis is significant because it helps to identify
which input parameters are most critical to achieving good memory performance. By analyzing the sensitivity of the system to fluctuations in these
parameters, researchers can gain insights into how to optimize the system's design and operation to achieve better performance. Additionally, by
understanding which parameters are most sensitive, researchers can develop strategies to mitigate the impact of experimental drift or other sources of
variability on the system's performance. Overall, memory sensitivity analysis is a valuable tool for optimizing quantum memory systems and ensuring
their reliability and robustness.


Sources:
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf
/content/papers/Variance-based.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Experimental techniques used to analyze memory sensitivity
include:

1. Variance-based sensitivity analysis: This involves measuring the variance in memory performance due to changes in control field parameters, and
calculating the sensitivity of the memory to these changes.
2. Single-parameter sensitivity calculations: This involves analyzing the sensitivity of the memory to changes in a single parameter at a time, such
as pulse area, delay, or pulse duration.
3. Two- and three-parameter sensitivity calculations: These involve analyzing the sensitivity of the memory to correlations between multiple
parameters, such as delays and pulse areas.
4. Upper bounds on memory fidelity: These involve calculating an upper bound on the sensitivity of the memory to experimental fluctuations and drift.
5. Lower bounds on ﬁdelity sensitivity: These involve calculating a lower bound on the sensitivity of the memory to experimental fluctuations and
drift.


Sources:
/content/papers/Variance-based.pd

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Gaussian control field has a direct impact on the memory
performance. When the control field parameters, such as the pulse area, delay, and duration, deviate from their optimal values, it leads to
fluctuations in the memory efficiency. These fluctuations can result in a decrease in the overall memory performance.

To quantify the sensitivity of the memory performance to these fluctuations, we can use the concept of "overlap fidelity." This measure calculates the
similarity between the optimal control fields at nearby points in the memory parameter space. The regions with the lowest overlap fidelity are those
that are most sensitive to fluctuations in the memory parameters.

As shown in Figure 3(c), the region corresponding to the absorb-then-transfer protocol has the lowest overlap fidelity, indicating that it is the most
sensitive to fluctuations in the memory parameters. Additionally, the dependence of memory efficiency fluctuations on memory parameter fluctuations is
plotted i

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the focus of variance-based sensitivity analysis in quantum memory?
 response:   The focus of variance-based sensitivity analysis in
quantum memory is todetermine the sensitivity of a quantum memory implementation toexperimental fluctuations and drift, particularly in terms of
itsperformance metric, such as memory efficiency.


Q1: What does Λ-type quantum memory refer to?
 response:   In the context of the paper " Variance-based sensitivity analysis of /Lambda1-type quantum
memory," /Lambda1-type quantum memory refers to a specific type of optical quantum memory that utilizes the /Lambda1 resonance transition in an atomic
ensemble to store and retrieve photonic quantum states.


Q2: Why are control field parameters crucial in Λ-type quantum memory systems?
 response:   Control field parameters are crucial in Λ-type quantum
memory systems because they directly affect the efficiency of the memory process. The control fields are used to manipulate the quantum states of the
at

In [ ]:
json_file_name = 'VarianceQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'VarianceQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)